In [ ]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np

In [ ]:
def kin_time_invariant(p, f, birth_female = 1/2.04, pi = None, output_kin = None):

    # Make matrix transition from vectors
    ages = len(p)
    age = np.arange(ages)
    Ut = np.zeros((ages, ages))
    Mt = np.zeros((ages, ages))
    zeros = np.zeros((ages, ages))

    # Fill the transition matrices
    for i in range(ages-1):
        Ut[i+1, i] = p[i]

    Ut[ages-1, ages-1] = p[ages-1]
    np.fill_diagonal(Mt, 1 - np.array(p))
    Ut = np.block([
        [Ut, zeros],
        [Mt, zeros]
    ])

    # Fertility matrix initialization
    ft = np.zeros((ages * 2, ages * 2))
    ft[0, :ages] = np.array(f) * birth_female

    # Create a matrix of zeros with dimensions (ages * 2, ages * 2)
    e = np.zeros((ages * 2, ages * 2))

    # Set the diagonal of the first ages x ages block to 1
    np.fill_diagonal(e[:ages, :ages], 1)


    # Creating pi———NEEDS AN IF STATEMENT LATER TO ACCOUNT FOR FUNCTION ARGUMENT
    A = Ut[0:ages, 0:ages] + ft[0:ages, 0:ages]
    eigenvalues, eigenvectors = np.linalg.eig(A)
    lambda_ = np.real(eigenvalues[0])
    w = np.real(eigenvectors[:, 45]) # WEIRD——45th eigenvector needs to be
    # pulled in order to match the results from the GitHub. May be due to
    # np.linalg.eig() (Python) and eigen() (r) using different algorithms that
    # return the calculated eigenvectors in a different order.
    w = w / np.sum(w)
    pi = w * A[0, :] / np.sum(w * A[0, :])
    pi = pi.reshape(-1)

    # Initialize the all individual arrays for all the kin as zero matrices
    d = np.zeros((ages * 2, ages))
    gd = np.zeros((ages * 2, ages))
    ggd = np.zeros((ages * 2, ages))
    m = np.zeros((ages * 2, ages))
    gm = np.zeros((ages * 2, ages))
    ggm = np.zeros((ages * 2, ages))
    os = np.zeros((ages * 2, ages))
    ys = np.zeros((ages * 2, ages))
    nos = np.zeros((ages * 2, ages))
    nys = np.zeros((ages * 2, ages))
    oa = np.zeros((ages * 2, ages))
    ya = np.zeros((ages * 2, ages))
    coa = np.zeros((ages * 2, ages))
    cya = np.zeros((ages * 2, ages))

    # initialize special matrix for mother
    m[:len(pi), 0] = pi

    # Kin "propagation" throughout Focal's life. Reference Caswell I for formulas
    for i in range(0, ages - 1):
        d[:, i+1] = np.dot(Ut, d[:, i]) + np.dot(ft, e[:, i])
        gd[:, i+1] = np.dot(Ut, gd[:, i]) + np.dot(ft, d[:, i])
        ggd[:, i+1] = np.dot(Ut, ggd[:, i]) + np.dot(ft, gd[:, i])
        m[:, i+1] = np.dot(Ut, m[:, i])
        ys[:, i+1] = np.dot(Ut, ys[:, i]) + np.dot(ft, m[:, i])
        nys[:, i+1] = np.dot(Ut, nys[:, i]) + np.dot(ft, ys[:, i])

    gm[:ages, 0] = np.dot(m[:ages, :], pi)
    for i in range(0, ages - 1):
        gm[:, i+1] = np.dot(Ut, gm[:, i])

    ggm[:ages, 0] = np.dot(gm[:ages, :], pi)
    for i in range(0, ages - 1):
        ggm[:, i+1] = np.dot(Ut, ggm[:, i])

    os[:ages, 0] = np.dot(d[:ages, :], pi)
    nos[:ages, 0] = np.dot(gd[:ages, :], pi)
    for i in range(0, ages - 1):
        os[:, i+1] = np.dot(Ut, os[:, i])
        nos[:, i+1] = np.dot(Ut, nos[:, i]) + np.dot(ft, os[:, i])

    oa[:ages, 0] = np.dot(os[:ages, :], pi)
    ya[:ages, 0] = np.dot(ys[:ages, :], pi)
    coa[:ages, 0] = np.dot(nos[:ages, :], pi)
    cya[:ages, 0] = np.dot(nys[:ages, :], pi)
    for i in range(0, ages - 1):
        oa[:, i+1] = np.dot(Ut, oa[:, i])
        ya[:, i+1] = np.dot(Ut, ya[:, i]) + np.dot(ft, gm[:, i])
        coa[:, i+1] = np.dot(Ut, coa[:, i]) + np.dot(ft, oa[:, i])
        cya[:, i+1] = np.dot(Ut, cya[:, i]) + np.dot(ft, ya[:, i])

    # Creating the kin_dict dictionary; each entry holds the matrix containing
    # kin information
    kin_dict = {
        'd': d,
        'gd': gd,
        'ggd': ggd,
        'm': m,
        'gm': gm,
        'ggm': ggm,
        'os': os,
        'ys': ys,
        'nos': nos,
        'nys': nys,
        'oa': oa,
        'ya': ya,
        'coa': coa,
        'cya': cya
    }

    # Deleting all other entries except for the ones specified in output_kin
    if output_kin is not None:
        for key in list(kin_dict.keys()):
            if key not in output_kin:
                del kin_dict[key]

    # Align the bottom half of each matrix (representing dead kin) with the top half
    for key, matrix in kin_dict.items():
        matrix[ages:2*ages, :ages-1] = matrix[ages:2*ages, 1:ages]
        matrix[ages:2*ages, ages-1] = 0
        kin_dict[key] = matrix

    # Create lists to hold the output data
    kin_ages = []
    focal_ages = []
    living_kin = []
    dead_kin = []

    # Initialize an empty DataFrame to store the results
    output = pd.DataFrame()

    # Iterate through each matrix in kin_dict and process it
    for kin_name, matrix in kin_dict.items():
        # Create lists to hold the output data
        kin_type = []
        kin_ages = []
        focal_ages = []
        living_kin = []
        dead_kin = []

        # Iterate through each age of the kin and Focal
        for kin_age in range(ages):
            for focal_age in range(ages):
                kin_type.append(kin_name)
                kin_ages.append(kin_age)
                focal_ages.append(focal_age)
                living_kin.append(matrix[kin_age, focal_age])
                dead_kin.append(matrix[kin_age + ages, focal_age])

        # Create a DataFrame for the current kin type
        kin_df = pd.DataFrame({
            "Kin": kin_type,
            'Age of Kin': kin_ages,
            'Age of Focal': focal_ages,
            'Number of Living Kin': living_kin,
            'Number of Dead Kin': dead_kin
        })

        # Append the current kin DataFrame to the output DataFrame
        output = pd.concat([output, kin_df], ignore_index=True)

    return output









# Assuming swe_surv_df and swe_fert_df are already read from CSV
# NOTE: TO MAKE THIS RUN, DATASETS NEEDED TO BE ADDED TO THE "CONTENT" FOLDER

swe_surv_df = pd.read_csv('swe_surv_2015.csv', names=['age', 'surv_prob'], skiprows=1)
swe_fert_df = pd.read_csv('swe_asfr_2015.csv', names=['age', 'fert_prob'], skiprows=1)

p = swe_surv_df['surv_prob'].to_list()
f = swe_fert_df['fert_prob'].to_list()

result = kin_time_invariant(p, f)

result.to_csv('output.csv', index = False)

